In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import datetime as dt

import numpy as np
import pandas as pd

from deep_hedging import Ticker, Underlyings, NelsonSiegelCurve
from deep_hedging.monte_carlo import GBMPricer
from deep_hedging.non_linear.exotic.basket.worst_of_digital import WorstOfDigitalCall

RANDOM_SEED = 12
PATH = Path("../../data/")

In [3]:
N_YEARS = 5
N_PATHS = 1_000

FIXING = dt.datetime(2021, 4, 19)
FINAL_FIXING = dt.datetime(2024, 4, 17)

In [4]:
underlyings = Underlyings(
    tickers=[
        Ticker("Sanofi", "SAN.PA"),
        Ticker("GlaxoSmithKline PLC", "GSK.L"),
        Ticker("Bayer AG", "BAYN.DE"),
    ],
    start=N_YEARS,
    end=FIXING,
)

[*********************100%%**********************]  3 of 3 completed


In [5]:
nss_params = pd.read_csv(PATH / "rub_nss.csv", sep=";")
rub_params = nss_params[
    nss_params["tradedate"] == f"{FIXING.strftime('%d.%m.%Y')}"
].loc[:, ["B1", "B2", "B3", "T1"]]
rub_params = rub_params.replace(",", ".", regex=True).astype(float)
rub_params

,B1,B2,B3,T1
1833,744.953996,-273.365215,-271.811557,0.905105


In [6]:
curve = NelsonSiegelCurve(
    b0=rub_params.B1.values[0] / 100,
    b1=rub_params.B2.values[0] / 100,
    b2=rub_params.B3.values[0] / 100,
    tau=rub_params.T1.values[0] / 100,
)
curve.get_rate(1.0)

0.07400815995797354

In [7]:
worst_of_digital = WorstOfDigitalCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    frequency=0.5,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    digital_coupon=0.05,
)
price = worst_of_digital.price(spot=[1.0, 1.0, 1.0])
print(f"{price:,.4f}")

0.0433
